# SUJET: Analyser les données du site e-commerce pour identifier les tendances, les comportements des utilisateurs, et les opportunités d'amélioration

## IMPORTER LES LIBRAIRIES

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
import pandas as pd  #manipulation des donnees
import numpy as np   #calcul mathematiques
import seaborn as sns   #visualiser les données
import matplotlib.pyplot as plt  #visualisation
import scipy as scipy   # faire des statistques et des probabilités
import warnings         #gestion des avertissements
warnings.filterwarnings('ignore')

## ETABLIR LE RESEAU EN LOCAL

In [3]:
# Forcer l'utilisation de localhost, pour éviter les erreurs de configuration réseau
os.environ['SPARK_LOCAL_IP'] = 'localhost' 

#### Compréhension des variables

timestamp: valeur numérique qui représente un instant précis dans le temps, plus sous cette forme 2025-02-15 14:30:00

visitorid: l'id des visiteurs

event: Vue, carte ajoutée dans le panier, transaction

transactionid: l'id de la transaction

categoryid: l'id de la categories

parentid: l'id des parents

itemid: l'id de l'article

property: propriétés des articles

## INITIALISATION D'UNE SESSION SPARK

In [4]:
spark = (SparkSession.builder
    .appName("AppEcommerce") 
    .master("local[4]") # 4 cœurs, donc 50% du nombre de mes coeurs
    .config("spark.driver.memory", "8G") # 50% de ma RAM
    .config("spark.sql.shuffle.partitions", "100") # Adapté pour 4 cœurs
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/18 18:49:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Vérifier la mémoire allouée
print("Mémoire allouée RAM :", spark.sparkContext.getConf().get("spark.driver.memory"))  # Doit afficher

Mémoire allouée RAM : 8G


## IMPORTER LES DONNEES

#### DATA EVENTS

In [ ]:
# Lire le fichier csv
data_events = spark.read.csv(r"C:\Users\Эли Жоэль\ANALYSE_DE_PERFORMANCES_ET_OPTIMISATION_DE_SITE_E-COMMERCE\data\transformed\events.csv", 
                   header=True,         # Si le fichier a des en-têtes
                   inferSchema=True,)   # Détecte les types (sinon tout en string)

In [7]:
data_events.show()

+-------------+---------+---------+------+-------------+
|    timestamp|visitorid|    event|itemid|transactionid|
+-------------+---------+---------+------+-------------+
|1433221332117|   257597|     view|355908|         NULL|
|1433224214164|   992329|     view|248676|         NULL|
|1433221999827|   111016|     view|318965|         NULL|
|1433221955914|   483717|     view|253185|         NULL|
|1433221337106|   951259|     view|367447|         NULL|
|1433224086234|   972639|     view| 22556|         NULL|
|1433221923240|   810725|     view|443030|         NULL|
|1433223291897|   794181|     view|439202|         NULL|
|1433220899221|   824915|     view|428805|         NULL|
|1433221204592|   339335|     view| 82389|         NULL|
|1433222162373|   176446|     view| 10572|         NULL|
|1433221701252|   929206|     view|410676|         NULL|
|1433224229496|    15795|     view| 44872|         NULL|
|1433223697356|   598426|     view|156489|         NULL|
|1433224078165|   223343|     v

In [8]:
# voir les types de colonnes
data_events.dtypes

[('timestamp', 'bigint'),
 ('visitorid', 'int'),
 ('event', 'string'),
 ('itemid', 'int'),
 ('transactionid', 'int')]

#### DATA_CATEGORY

In [ ]:
data_category = spark.read.csv(r"C:\Users\Эли Жоэль\ANALYSE_DE_PERFORMANCES_ET_OPTIMISATION_DE_SITE_E-COMMERCE\data\transformed\category_tree.csv", 
                   header=True,       
                   inferSchema=True,)

In [10]:
data_category.show()

+----------+--------+
|categoryid|parentid|
+----------+--------+
|      1016|     213|
|       809|     169|
|       570|       9|
|      1691|     885|
|       536|    1691|
|       231|    NULL|
|       542|     378|
|      1146|     542|
|      1140|     542|
|      1479|    1537|
|        83|    1621|
|       688|     893|
|       257|     312|
|      1640|     622|
|       963|    1281|
|       412|    1110|
|       948|    1110|
|       934|    1110|
|       148|    1110|
|        12|    1110|
+----------+--------+
only showing top 20 rows



In [12]:
# voir les types de colonnes
data_category.dtypes

[('categoryid', 'int'), ('parentid', 'int')]

#### DATA_ITEM_PROPERTIES_1

In [ ]:
data_item_properties_1=spark.read.csv(r"C:\Users\Эли Жоэль\ANALYSE_DE_PERFORMANCES_ET_OPTIMISATION_DE_SITE_E-COMMERCE\data\transformed\item_properties_part1.csv",
                                      header=True,
                                      inferSchema=True)

In [14]:
data_item_properties_1.show()

+-------------+------+----------+--------------------+
|    timestamp|itemid|  property|               value|
+-------------+------+----------+--------------------+
|1435460400000|460429|categoryid|                1338|
|1441508400000|206783|       888|1116713 960601 n2...|
|1439089200000|395014|       400|n552.000 639502 n...|
|1431226800000| 59481|       790|          n15360.000|
|1431831600000|156781|       917|              828513|
|1436065200000|285026| available|                   0|
|1434250800000| 89534|       213|             1121373|
|1431831600000|264312|         6|              319724|
|1433646000000|229370|       202|             1330310|
|1434250800000| 98113|       451|     1141052 n48.000|
|1439089200000|450113|       888|1038400 45956 n50...|
|1435460400000|244127|       400|n552.000 639502 n...|
|1439694000000|264319|       227|      1283144 353870|
|1439694000000|348323|       839|     1026952 1162729|
|1434250800000|169055|       790|          n21000.000|
|143727480

#### DATA_ITEM_PROPERTIES_2

In [ ]:
data_item_properties_2=spark.read.csv(r"C:\Users\Эли Жоэль\ANALYSE_DE_PERFORMANCES_ET_OPTIMISATION_DE_SITE_E-COMMERCE\data\transformed\item_properties_part2.csv",
                                      header=True,
                                      inferSchema=True)

In [16]:
data_item_properties_2.show()

+-------------+------+----------+--------------------+
|    timestamp|itemid|  property|               value|
+-------------+------+----------+--------------------+
|1433041200000|183478|       561|              769062|
|1439694000000|132256|       976|     n26.400 1135780|
|1435460400000|420307|       921|     1149317 1257525|
|1431831600000|403324|       917|             1204143|
|1435460400000|230701|       521|              769062|
|1433041200000|286407|       202|              820407|
|1438484400000|256368|       888|437265 1296497 n2...|
|1437879600000|307534|       888|150169 212349 109...|
|1439089200000|102767|       888|5135 790941 10558...|
|1431831600000|215180|        71|             1096621|
|1435460400000|319117|       202|              876900|
|1433041200000|377946|       202|235534 307222 237...|
|1432436400000|177411|       663|            n204.000|
|1435460400000| 27497|        71|             1115859|
|1431831600000|430773|       283|257131 82777 4938...|
|143122680

#### DATA_ITEM_PROPERTIES

In [17]:
# Concaténation (union des tables avec mêmes colonnes)des deux item_properties 1 et 2
data_item_properties=data_item_properties_1.union(data_item_properties_2)
data_item_properties.show()

+-------------+------+----------+--------------------+
|    timestamp|itemid|  property|               value|
+-------------+------+----------+--------------------+
|1435460400000|460429|categoryid|                1338|
|1441508400000|206783|       888|1116713 960601 n2...|
|1439089200000|395014|       400|n552.000 639502 n...|
|1431226800000| 59481|       790|          n15360.000|
|1431831600000|156781|       917|              828513|
|1436065200000|285026| available|                   0|
|1434250800000| 89534|       213|             1121373|
|1431831600000|264312|         6|              319724|
|1433646000000|229370|       202|             1330310|
|1434250800000| 98113|       451|     1141052 n48.000|
|1439089200000|450113|       888|1038400 45956 n50...|
|1435460400000|244127|       400|n552.000 639502 n...|
|1439694000000|264319|       227|      1283144 353870|
|1439694000000|348323|       839|     1026952 1162729|
|1434250800000|169055|       790|          n21000.000|
|143727480

In [18]:
# Voir les types de colonne
data_item_properties.dtypes

[('timestamp', 'bigint'),
 ('itemid', 'int'),
 ('property', 'string'),
 ('value', 'string')]

In [42]:
# Pour filtrer les événements par vue
eli.filter(eli["property"] == "categoryid").show()

[Stage 95:>                                                         (0 + 1) / 1]

+------+-------------+----------+-----+---------+-----+-------------+
|itemid|    timestamp|  property|value|visitorid|event|transactionid|
+------+-------------+----------+-----+---------+-----+-------------+
| 13810|1433041200000|categoryid| 1421|     NULL| NULL|         NULL|
| 35575|1435460400000|categoryid| 1059|     NULL| NULL|         NULL|
| 47849|1431226800000|categoryid| 1510|     NULL| NULL|         NULL|
| 49832|1433646000000|categoryid| 1167|     NULL| NULL|         NULL|
| 50588|1431226800000|categoryid| 1592|     NULL| NULL|         NULL|
| 57396|1431831600000|categoryid|  914|     NULL| NULL|         NULL|
| 72725|1431226800000|categoryid|  342|     NULL| NULL|         NULL|
| 76151|1440903600000|categoryid| 1085|     NULL| NULL|         NULL|
| 76417|1441508400000|categoryid| 1244|     NULL| NULL|         NULL|
| 80067|1431226800000|categoryid| 1142|     NULL| NULL|         NULL|
|115407|1439089200000|categoryid|  491|     NULL| NULL|         NULL|
|119360|143908920000

In [43]:
eli.select('event').distinct().show()

[Stage 100:=============================================>           (4 + 1) / 5]

+-----+
|event|
+-----+
| NULL|
+-----+



## ANALYSE DU COMPORTEMENT UTILISATEUR 

#### Pour analyser le <font color="red">comportement utilisateur <font color="black"> nous allons plus nous intéresser à la table <font color="red">DATA_EVENTS <font color="black"> qui comporte les colonnes: <font color="red">temps, visiteur, article,  évènements effectués et transactions.<font color="black"> C'est avec ces informations de colonne que nous pouvons effectuer les analyses de comportement utilisateur, contrairement aux autres tables qui informent plus sur les catégories d'articles et leurs valeurs.

### <font color="green"> TEMPS PASSE SUR CHAQUE EVENEMENT

#### On va créer une table <font color="red"> USERS_TIME <font color="black"> à partir de la table <font color="red"> DATA_EVENTS <font color="black"> et utiliser les colonnes: <font color="red"> 'timestamp','visitorid'<font color="black"> et <font color="red">'itemid'.<font color="black"> Pourquoi utiliser toutes ces colonnes, parce que pour un site E-Commerce ces colonnes sont importantes.

In [20]:
# Fenêtre : groupé par visitorid (et éventuellement transactionid si tu veux isoler les sessions d'achat),
# trié par timestamp
windowSpec = Window.partitionBy("visitorid").orderBy("timestamp")

# Récupérer le timestamp de l'événement suivant pour ce visiteur
users_time = data_events.withColumn(
    "next_timestamp", F.lead("timestamp").over(windowSpec)
)

# Calculer le temps passé (secondes)
users_time = users_time.withColumn(
    "time_spent_seconds",
    (F.col("next_timestamp") - F.col("timestamp")) / 1000
)

# Afficher le résultat
users_time.show()

[Stage 16:>                                                         (0 + 1) / 1]

+-------------+---------+-----+------+-------------+--------------+------------------+
|    timestamp|visitorid|event|itemid|transactionid|next_timestamp|time_spent_seconds|
+-------------+---------+-----+------+-------------+--------------+------------------+
|1442352267167|        4| view|177677|         NULL|          NULL|              NULL|
|1431581976753|        7| view|139394|         NULL| 1431582162817|           186.064|
|1431582162817|        7| view|164941|         NULL| 1431750039214|        167876.397|
|1431750039214|        7| view|226353|         NULL|          NULL|              NULL|
|1433030513812|        8| view|434230|         NULL|          NULL|              NULL|
|1438713029611|       10| view|248766|         NULL|          NULL|              NULL|
|1430668212744|       12| view| 70225|         NULL|          NULL|              NULL|
|1433650323043|       23| view| 44608|         NULL| 1434075948023|         425624.98|
|1434075948023|       23| view|283916|     

#### <font color="green">Les différents évènements

In [24]:
users_time.select('event').distinct().show()

[Stage 22:==============>                                           (1 + 3) / 4]

+-----------+
|      event|
+-----------+
|transaction|
|  addtocart|
|       view|
+-----------+



#### <font color="green">Temps d'utilisateurs par transaction 

In [27]:
# filtrer les événements par transaction et conversion de "time_spent_seconds" en "time_spent"
users_time_transaction = (
    users_time
    .filter(users_time["event"].isin(["transaction", "time_spent_seconds"]))
    .withColumn("hours", (F.col("time_spent_seconds") / 3600).cast("int"))
    .withColumn("minutes", ((F.col("time_spent_seconds") % 3600) / 60).cast("int"))
    .withColumn("seconds", (F.col("time_spent_seconds") % 60).cast("int"))
    .withColumn(
        "time_spent",
        F.format_string("%02d:%02d:%02d", F.col("hours"), F.col("minutes"), F.col("seconds"))
    )
    .orderBy(F.col("time_spent_seconds").desc())  # tri correct sur la durée réelle
    .select("timestamp", "visitorid", "event", "itemid", "transactionid",
            "next_timestamp", "time_spent")
)
users_time_transaction.show(truncate=False)

[Stage 32:==============>                                           (1 + 3) / 4]

+-------------+---------+-----------+------+-------------+--------------+----------+
|timestamp    |visitorid|event      |itemid|transactionid|next_timestamp|time_spent|
+-------------+---------+-----------+------+-------------+--------------+----------+
|1430713169917|243592   |transaction|295613|9810         |1442352882196 |3233:15:12|
|1430666959685|729806   |transaction|317436|2410         |1442195166046 |3202:16:46|
|1431564765536|1027767  |transaction|434277|10818        |1442508504828 |3039:55:39|
|1431031797329|661382   |transaction|112831|7183         |1441930872025 |3027:31:14|
|1431376601896|1058213  |transaction|32581 |9007         |1442198633624 |3006:07:11|
|1430627474950|266417   |transaction|301359|12546        |1440868157041 |2844:38:02|
|1432446065373|1117333  |transaction|42002 |6831         |1442467815769 |2783:49:10|
|1431903675073|1030794  |transaction|193828|2632         |1441575298444 |2686:33:43|
|1431830495979|988412   |transaction|449757|10900        |1441406

##### CONSTAT: <font color="red">On peut remarquer que ceux qui ont effectué une transaction ont beaucoup passé de temps sur le site.

### <font color="green">Temps d'utiliseurs par addtocart

In [28]:
# filtrer les événements par addtocart
users_time_transaction = (
    users_time
    .filter(users_time["event"].isin(["addtocart", "time_spent_seconds"]))
    .withColumn("hours", (F.col("time_spent_seconds") / 3600).cast("int"))
    .withColumn("minutes", ((F.col("time_spent_seconds") % 3600) / 60).cast("int"))
    .withColumn("seconds", (F.col("time_spent_seconds") % 60).cast("int"))
    .withColumn(
        "time_spent",
        F.format_string("%02d:%02d:%02d", F.col("hours"), F.col("minutes"), F.col("seconds"))
    )
    .orderBy(F.col("time_spent_seconds").desc())  # tri correct sur la durée réelle
    .select("timestamp", "visitorid", "event", "itemid", "transactionid",
            "next_timestamp", "time_spent")
)
users_time_transaction.show(truncate=False)

[Stage 37:>                                                         (0 + 4) / 4]

+-------------+---------+---------+------+-------------+--------------+----------+
|timestamp    |visitorid|event    |itemid|transactionid|next_timestamp|time_spent|
+-------------+---------+---------+------+-------------+--------------+----------+
|1430812534038|1172312  |addtocart|293279|NULL         |1442255935756 |3178:43:21|
|1431308539025|151125   |addtocart|236203|NULL         |1442460370043 |3097:43:51|
|1430840723540|1241044  |addtocart|159174|NULL         |1441859641956 |3060:48:38|
|1431973757238|292557   |addtocart|354233|NULL         |1442518797364 |2929:10:40|
|1432057670268|346506   |addtocart|276704|NULL         |1442463913574 |2890:37:23|
|1431448054472|996698   |addtocart|331510|NULL         |1441722228166 |2853:56:13|
|1431888758736|1205949  |addtocart|379432|NULL         |1441317947063 |2619:13:08|
|1432777452200|631732   |addtocart|157725|NULL         |1442159830340 |2606:12:58|
|1432016623960|2004     |addtocart|132772|NULL         |1441321625904 |2584:43:21|
|143

##### CONSTAT: <font color="red">On peut remarquer que ceux qui ont ajouter la carte n'ont effectué aucune transaction mais ont beaucoup passé de temps sur le site.

#### <font color="green">Temps d'utilisateurs par vue

In [29]:
# Pour filtrer les événements par vue
# filtrer les événements par addtocart
users_time_transaction = (
    users_time
    .filter(users_time["event"].isin(["view", "time_spent_seconds"]))
    .withColumn("hours", (F.col("time_spent_seconds") / 3600).cast("int"))
    .withColumn("minutes", ((F.col("time_spent_seconds") % 3600) / 60).cast("int"))
    .withColumn("seconds", (F.col("time_spent_seconds") % 60).cast("int"))
    .withColumn(
        "time_spent",
        F.format_string("%02d:%02d:%02d", F.col("hours"), F.col("minutes"), F.col("seconds"))
    )
    .orderBy(F.col("time_spent_seconds").desc())  # tri correct sur la durée réelle
    .select("timestamp", "visitorid", "event", "itemid", "transactionid",
            "next_timestamp", "time_spent")
)
users_time_transaction.show(truncate=False)

[Stage 40:>                                                         (0 + 4) / 4]

+-------------+---------+-----+------+-------------+--------------+----------+
|timestamp    |visitorid|event|itemid|transactionid|next_timestamp|time_spent|
+-------------+---------+-----+------+-------------+--------------+----------+
|1430754061469|35852    |view |304774|NULL         |1442541512487 |3274:17:31|
|1430683624408|147749   |view |193776|NULL         |1442436610946 |3264:43:06|
|1430863641839|228468   |view |72062 |NULL         |1442525591865 |3239:25:50|
|1430924817945|593446   |view |408172|NULL         |1442528565686 |3223:15:47|
|1430775400802|608895   |view |372845|NULL         |1442376282462 |3222:28:01|
|1430703422555|70725    |view |306419|NULL         |1442303818385 |3222:19:55|
|1430807603515|506412   |view |350629|NULL         |1442387381643 |3216:36:18|
|1430800415368|597960   |view |51947 |NULL         |1442376945644 |3215:42:10|
|1430846105727|321238   |view |264392|NULL         |1442421764175 |3215:27:38|
|1430678763264|757516   |view |264746|NULL         |

##### CONSTAT: <font color="red">On peut remarquer que ceux qui ont vu les articles n'ont effectué aucune transaction mais ont beaucoup passé de temps sur le site.

ANALYSE:

#### On comprend par analyse qu'un client peut voir un article et même ajouter un article au panier, ça ne garantit pas l'achat de l'article.

## ANALYSE SUR LA CATEGORIE D'ARTICLES

#### <font color="green"> TEMPS PASSE SUR CHAQUE CATEGORIE D'ARTICLES DU SITE 

In [30]:
# Jointure avec la table data_item_properties et users_time
item_properties_time = data_item_properties.join(
    users_time,
    on=["itemid","timestamp"],
    how="left"
)
# Afficher item_properties_time
item_properties_time.show()

[Stage 48:>                                                         (0 + 1) / 1]

+------+-------------+----------+--------------------+---------+-----+-------------+--------------+------------------+
|itemid|    timestamp|  property|               value|visitorid|event|transactionid|next_timestamp|time_spent_seconds|
+------+-------------+----------+--------------------+---------+-----+-------------+--------------+------------------+
|  7913|1435460400000|       888|       618068 253573|     NULL| NULL|         NULL|          NULL|              NULL|
| 23287|1436670000000| available|                   0|     NULL| NULL|         NULL|          NULL|              NULL|
| 26065|1435460400000|       790|         n193200.000|     NULL| NULL|         NULL|          NULL|              NULL|
| 26515|1431226800000|         6|             1103229|     NULL| NULL|         NULL|          NULL|              NULL|
| 33864|1439694000000|       790|          n28068.000|     NULL| NULL|         NULL|          NULL|              NULL|
| 59481|1431226800000|       790|          n1536

#### <font color="green">Produits disponibles 

In [33]:
# Filtrer les propriétés par article disponible
product_available= item_properties_time.filter(item_properties_time["property"] == "available")
product_available.show()

[Stage 60:>                                                         (0 + 4) / 4]

+------+-------------+---------+-----+---------+-----+-------------+--------------+------------------+
|itemid|    timestamp| property|value|visitorid|event|transactionid|next_timestamp|time_spent_seconds|
+------+-------------+---------+-----+---------+-----+-------------+--------------+------------------+
|  9744|1440298800000|available|    1|     NULL| NULL|         NULL|          NULL|              NULL|
| 20026|1442113200000|available|    0|     NULL| NULL|         NULL|          NULL|              NULL|
| 23287|1436670000000|available|    0|     NULL| NULL|         NULL|          NULL|              NULL|
|100993|1431226800000|available|    0|     NULL| NULL|         NULL|          NULL|              NULL|
|101887|1437879600000|available|    0|     NULL| NULL|         NULL|          NULL|              NULL|
|102974|1431831600000|available|    1|     NULL| NULL|         NULL|          NULL|              NULL|
|106601|1431831600000|available|    0|     NULL| NULL|         NULL|     

In [36]:
# Nombre d'articles disponibles
nbr_product_available= product_available.filter(product_available["value"] == 1).count()
print(f"Le nombre d'articles disponibles est :{nbr_product_available}")

[Stage 78:>                                                         (0 + 4) / 4]

Le nombre d'articles disponibles est :640553


In [37]:
# voir les valeurs uniques "product_available"
product_available.select('transactionid').distinct().show()

[Stage 87:>                                                         (0 + 4) / 5]

+-------------+
|transactionid|
+-------------+
|         NULL|
+-------------+



ANALYSE:

#### Aucun article disponible n'a été acheté et n'a reçu aucune visite donc <font color="red">nous comprenons que ces articles n'attirent nullement l'attention.

# TEST A/B